# 02 - Data Processing

In [1]:
import sys
sys.executable

'/usr/local/bin/python'

## Imports

In [2]:
import numpy as np
import pandas as pd

import stock_utils.constants as cnst
import stock_utils.data as sd

pd.set_option('display.max_columns', None)

NSE_DATA_DIR = PosixPath('../data/NSE') | Valid: True
SCREENER_EPS_DATA_DIR = PosixPath('../data/Screener.in/EPS') | Valid: True
PROCESSED_DATA_DIR = PosixPath('../data/processed') | Valid: True


## Constants

In [3]:
stock_symbols = sd.get_all_stock_symbols(
    cnst.NSE_DATA_DIR
)

stock_symbols

['DEEPAKFERT', 'HDFCBANK', 'INDIGOPNTS', 'ITBEES', 'JUBLFOOD']

In [4]:
STOCK_SYMBOL = stock_symbols[2]
STOCK_SYMBOL

'INDIGOPNTS'

## Data loading

In [5]:
stock_df = pd.read_parquet(cnst.NSE_DATA_DIR.joinpath(STOCK_SYMBOL, "consolidated.parquet"))
stock_df.columns = [
    'Date', 'Series',
    'Open', 'High', 'Low', 'Prev Close', 'LTP', 'Close',
    'VWAP', '52W H', '52W L',
    'Volume', 'Value', '#Trades'
] + (["PE"] if "PE" in stock_df.columns else [])
stock_df

,Date,Series,Open,High,Low,Prev Close,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,PE
0,2021-02-02,EQ,2607.50,3129.00,2436.05,1490.00,3129.00,3117.15,2684.35,3129.00,2436.05,12652036,3.396245e+10,812580,189.033
1,2021-02-03,EQ,3239.00,3329.95,2831.25,3117.15,2953.45,2924.25,3085.39,3329.95,2436.05,2955005,9.117329e+09,375333,177.335
2,2021-02-04,EQ,2948.85,2967.95,2853.00,2924.25,2866.00,2873.75,2902.67,3329.95,2436.05,476054,1.381829e+09,73464,174.272
3,2021-02-05,EQ,2866.00,2900.00,2603.00,2873.75,2621.00,2630.05,2694.77,3329.95,2436.05,587315,1.582680e+09,103675,159.494
4,2021-02-08,EQ,2590.00,2736.00,2551.00,2630.05,2700.00,2677.05,2666.44,3329.95,2436.05,507665,1.353659e+09,55033,162.344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770,2024-03-11,EQ,1384.45,1390.80,1363.00,1363.55,1368.00,1367.85,1372.64,1679.95,981.95,49564,6.803342e+07,10005,49.345
771,2024-03-12,EQ,1367.90,1374.70,1325.05,1367.85,1338.70,1337.20,1341.61,1679.95,981.95,57336,7.692242e+07,12955,48.240
772,2024-03-13,EQ,1337.20,1367.00,1303.05,1337.20,1310.50,1308.65,1313.57,1679.95,981.95,83612,1.098304e+08,14624,47.210
773,2024-03-14,EQ,1308.65,1352.40,1308.65,1308.65,1328.00,1329.00,1337.46,1679.95,981.95,78961,1.056072e+08,15931,47.944


In [6]:
stock_df.info(memory_usage = "deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 775 entries, 0 to 774
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        775 non-null    datetime64[ns]
 1   Series      775 non-null    object        
 2   Open        775 non-null    float64       
 3   High        775 non-null    float64       
 4   Low         775 non-null    float64       
 5   Prev Close  775 non-null    float64       
 6   LTP         775 non-null    float64       
 7   Close       775 non-null    float64       
 8   VWAP        775 non-null    float64       
 9   52W H       775 non-null    float64       
 10  52W L       775 non-null    float64       
 11  Volume      775 non-null    int64         
 12  Value       775 non-null    float64       
 13  #Trades     775 non-null    int64         
 14  PE          775 non-null    float64       
dtypes: datetime64[ns](1), float64(11), int64(2), object(1)
memory usage: 129.5

## Feature engineering

### Daily candle based columns

In [7]:
stock_df['Range'] = stock_df['High'] - stock_df['Low']
stock_df['IsGreen'] = (stock_df['Close'] >= stock_df['Prev Close']).astype(int)
stock_df['Is52WLow'] = np.isclose(stock_df['Low'], stock_df['52W L'], atol = 0, rtol = 1e-4).astype(int)
stock_df['Is52WHigh'] = np.isclose(stock_df['High'], stock_df['52W H'], atol = 0, rtol = 1e-4).astype(int)
stock_df = stock_df.drop(columns = ['Series', 'Prev Close'])
stock_df.sample(10)

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,PE,Range,IsGreen,Is52WLow,Is52WHigh
536,2023-03-31,1027.00,1030.00,1000.00,1009.00,1008.35,1016.22,1746.00,981.95,54660,5.554679e+07,5659,57.098,30.00,0,0,0
100,2021-06-30,2551.00,2555.00,2510.00,2515.00,2512.55,2527.92,3329.95,2220.25,26264,6.639321e+07,5021,168.854,45.00,0,0,0
288,2022-04-01,1610.95,1656.00,1605.05,1605.05,1616.80,1620.81,2740.00,1438.15,31155,5.049640e+07,5244,91.552,50.95,1,0,0
342,2022-06-21,1428.00,1482.00,1396.50,1400.00,1403.25,1420.58,2709.00,1353.60,23196,3.295185e+07,4825,79.459,85.50,0,0,0
526,2023-03-16,1066.10,1073.95,1041.20,1057.40,1054.90,1057.85,1746.00,991.25,115034,1.216884e+08,10764,59.734,32.75,0,0,0
131,2021-08-13,2429.00,2472.25,2405.25,2424.50,2422.15,2421.38,3329.95,2220.25,53247,1.289310e+08,11249,162.779,67.00,1,0,0
567,2023-05-19,1410.25,1418.00,1372.05,1403.00,1403.65,1395.09,1746.00,981.95,157422,2.196176e+08,13818,50.637,45.95,0,0,0
284,2022-03-28,1542.00,1545.85,1501.95,1517.00,1517.20,1517.86,2740.00,1438.15,31022,4.708708e+07,9117,101.962,43.90,0,0,0
708,2023-12-12,1474.95,1490.00,1461.20,1464.10,1469.30,1475.57,1679.95,981.95,44804,6.611141e+07,6820,53.005,28.80,0,0,0
402,2022-09-16,1599.00,1610.00,1569.00,1600.05,1601.35,1594.42,2695.00,1353.60,97078,1.547828e+08,9234,90.677,41.00,1,0,0


In [8]:
((stock_df[['IsGreen', 'Is52WLow', 'Is52WHigh']].sum() * 100) / len(stock_df)).round(2)

IsGreen      43.87
Is52WLow      7.87
Is52WHigh     0.26
dtype: float64

In [9]:
stock_df.shape

(775, 17)

### Streak

In [10]:
stock_df["Streak"] = (stock_df["IsGreen"] != stock_df["IsGreen"].shift(1)).cumsum()
stock_df["Streak"] = stock_df.groupby("Streak").cumcount() + 1
stock_df

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,PE,Range,IsGreen,Is52WLow,Is52WHigh,Streak
0,2021-02-02,2607.50,3129.00,2436.05,3129.00,3117.15,2684.35,3129.00,2436.05,12652036,3.396245e+10,812580,189.033,692.95,1,1,1,1
1,2021-02-03,3239.00,3329.95,2831.25,2953.45,2924.25,3085.39,3329.95,2436.05,2955005,9.117329e+09,375333,177.335,498.70,0,0,1,1
2,2021-02-04,2948.85,2967.95,2853.00,2866.00,2873.75,2902.67,3329.95,2436.05,476054,1.381829e+09,73464,174.272,114.95,0,0,0,2
3,2021-02-05,2866.00,2900.00,2603.00,2621.00,2630.05,2694.77,3329.95,2436.05,587315,1.582680e+09,103675,159.494,297.00,0,0,0,3
4,2021-02-08,2590.00,2736.00,2551.00,2700.00,2677.05,2666.44,3329.95,2436.05,507665,1.353659e+09,55033,162.344,185.00,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770,2024-03-11,1384.45,1390.80,1363.00,1368.00,1367.85,1372.64,1679.95,981.95,49564,6.803342e+07,10005,49.345,27.80,1,0,0,1
771,2024-03-12,1367.90,1374.70,1325.05,1338.70,1337.20,1341.61,1679.95,981.95,57336,7.692242e+07,12955,48.240,49.65,0,0,0,1
772,2024-03-13,1337.20,1367.00,1303.05,1310.50,1308.65,1313.57,1679.95,981.95,83612,1.098304e+08,14624,47.210,63.95,0,0,0,2
773,2024-03-14,1308.65,1352.40,1308.65,1328.00,1329.00,1337.46,1679.95,981.95,78961,1.056072e+08,15931,47.944,43.75,1,0,0,1


### Date based columns

In [11]:
stock_df['Day'] = stock_df['Date'].dt.day
stock_df['Month'] = stock_df['Date'].dt.month
stock_df['Year'] = stock_df['Date'].dt.year
stock_df['Weekday'] = stock_df['Date'].dt.weekday
stock_df['DayOfYear'] = stock_df['Date'].dt.day_of_year
stock_df['Quarter'] = stock_df['Date'].dt.quarter
stock_df['DaysSinceLastTradingSession'] = (stock_df['Date'] - stock_df['Date'].shift(1)).dt.days.fillna(1).astype(int)

stock_df.sample(10)

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,PE,Range,IsGreen,Is52WLow,Is52WHigh,Streak,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession
721,2024-01-01,1499.0,1504.95,1488.00,1492.00,1493.20,1494.28,1679.95,981.95,32780,4.898265e+07,5519,53.867,16.95,1,0,0,2,1,1,2024,0,1,1,3
175,2021-10-19,2495.0,2504.95,2470.00,2470.90,2474.50,2482.85,3329.95,2220.25,18405,4.569689e+07,5592,166.297,34.95,0,0,0,5,19,10,2021,1,292,4,1
131,2021-08-13,2429.0,2472.25,2405.25,2424.50,2422.15,2421.38,3329.95,2220.25,53247,1.289310e+08,11249,162.779,67.00,1,0,0,2,13,8,2021,4,225,3,1
436,2022-11-07,1550.4,1550.40,1470.00,1494.95,1492.55,1510.68,2395.00,1353.60,82040,1.239359e+08,11428,84.516,80.40,0,0,0,2,7,11,2022,0,311,4,3
197,2021-11-22,2287.0,2335.00,2251.00,2330.00,2319.90,2283.93,3329.95,2220.25,23384,5.340754e+07,7142,155.907,84.00,1,0,0,2,22,11,2021,0,326,4,4
710,2023-12-14,1473.0,1476.95,1462.00,1463.00,1464.60,1466.26,1679.95,981.95,41826,6.132788e+07,5684,52.835,14.95,0,0,0,8,14,12,2023,3,348,4,1
646,2023-09-11,1600.0,1605.00,1573.00,1579.05,1580.60,1585.65,1688.00,981.95,56001,8.879826e+07,8415,57.020,32.00,0,0,0,3,11,9,2023,0,254,3,3
406,2022-09-22,1575.0,1575.00,1513.65,1532.00,1528.85,1529.23,2695.00,1353.60,57594,8.807446e+07,11858,86.571,61.35,0,0,0,3,22,9,2022,3,265,3,1
9,2021-02-15,2600.0,2650.00,2565.05,2592.00,2595.65,2604.27,3329.95,2436.05,128961,3.358496e+08,16643,157.408,84.95,0,0,0,4,15,2,2021,0,46,1,3
540,2023-04-10,1097.5,1099.90,1066.25,1073.00,1071.35,1081.11,1746.00,981.95,43655,4.719586e+07,5799,38.649,33.65,0,0,0,1,10,4,2023,0,100,2,4


In [12]:
stock_df.shape

(775, 25)

### Moving averages

#### `Close`

In [13]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Close {window}MA'] = stock_df['Close'].rolling(window = window, min_periods = 1).mean().round(2)

stock_df.filter(regex = "Close.*")

,Close,Close 7MA,Close 15MA,Close 30MA,Close 60MA
0,3117.15,3117.15,3117.15,3117.15,3117.15
1,2924.25,3020.70,3020.70,3020.70,3020.70
2,2873.75,2971.72,2971.72,2971.72,2971.72
3,2630.05,2886.30,2886.30,2886.30,2886.30
4,2677.05,2844.45,2844.45,2844.45,2844.45
...,...,...,...,...,...
770,1367.85,1384.21,1415.54,1432.52,1454.13
771,1337.20,1375.11,1404.45,1428.82,1451.77
772,1308.65,1360.89,1392.22,1424.33,1448.86
773,1329.00,1352.70,1384.09,1421.24,1446.35


#### `Range`

In [14]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Range {window}MA'] = stock_df['Range'].rolling(window = window, min_periods = 1).mean().round(2)

stock_df.filter(regex = "Range.*")

,Range,Range 7MA,Range 15MA,Range 30MA,Range 60MA
0,692.95,692.95,692.95,692.95,692.95
1,498.70,595.82,595.82,595.82,595.82
2,114.95,435.53,435.53,435.53,435.53
3,297.00,400.90,400.90,400.90,400.90
4,185.00,357.72,357.72,357.72,357.72
...,...,...,...,...,...
770,27.80,29.64,30.68,39.25,36.58
771,49.65,33.87,32.08,39.41,36.96
772,63.95,40.86,33.64,40.77,37.78
773,43.75,43.00,33.70,40.83,38.26


#### `VWAP`

In [15]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'VWAP {window}MA'] = stock_df['VWAP'].rolling(window = window, min_periods = 1).mean().round(2)

stock_df.filter(regex = "VWAP.*")

,VWAP,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA
0,2684.35,2684.35,2684.35,2684.35,2684.35
1,3085.39,2884.87,2884.87,2884.87,2884.87
2,2902.67,2890.80,2890.80,2890.80,2890.80
3,2694.77,2841.80,2841.80,2841.80,2841.80
4,2666.44,2806.72,2806.72,2806.72,2806.72
...,...,...,...,...,...
770,1372.64,1385.05,1417.61,1436.67,1456.40
771,1341.61,1375.97,1406.83,1432.99,1454.17
772,1313.57,1362.40,1394.67,1428.78,1451.40
773,1337.46,1354.79,1386.48,1425.70,1449.10


#### `Volume`

In [16]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Volume {window}MA'] = stock_df['Volume'].rolling(window = window, min_periods = 1).mean().astype(int)

stock_df.filter(regex = "Volume.*")

,Volume,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA
0,12652036,12652036,12652036,12652036,12652036
1,2955005,7803520,7803520,7803520,7803520
2,476054,5361031,5361031,5361031,5361031
3,587315,4167602,4167602,4167602,4167602
4,507665,3435615,3435615,3435615,3435615
...,...,...,...,...,...
770,49564,35752,48634,95234,94734
771,57336,40624,49193,96071,93418
772,83612,51783,50356,97602,94142
773,78961,56997,49060,98710,94914


#### `Value`

In [17]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Value {window}MA'] = stock_df['Value'].rolling(window = window, min_periods = 1).mean().astype(int)

stock_df.filter(regex = "Value.*")

,Value,Value 7MA,Value 15MA,Value 30MA,Value 60MA
0,3.396245e+10,33962446981,33962446981,33962446981,33962446981
1,9.117329e+09,21539888009,21539888009,21539888009,21539888009
2,1.381829e+09,14820534938,14820534938,14820534938,14820534938
3,1.582680e+09,11511071310,11511071310,11511071310,11511071310
4,1.353659e+09,9479588872,9479588872,9479588872,9479588872
...,...,...,...,...,...
770,6.803342e+07,49339496,69434146,138641849,139018441
771,7.692242e+07,55664489,69656534,139646243,136947026
772,1.098304e+08,70248648,70379513,141498594,137787498
773,1.056072e+08,76898424,67839371,142840100,138744439


#### `#Trades`

In [18]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'#Trades {window}MA'] = stock_df['#Trades'].rolling(window = window, min_periods = 1).mean().astype(int)

stock_df.filter(regex = "#Trades.*")

,#Trades,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA
0,812580,812580,812580,812580,812580
1,375333,593956,593956,593956,593956
2,73464,420459,420459,420459,420459
3,103675,341263,341263,341263,341263
4,55033,284017,284017,284017,284017
...,...,...,...,...,...
770,10005,6126,7159,11886,10611
771,12955,7474,7595,12146,10688
772,14624,9435,7951,12407,10822
773,15931,10562,8061,12632,10994


### Target columns

In [19]:
for window in cnst.TARGET_WINDOWS:
    stock_df[f'Target {window}D'] = stock_df['Close'].shift(-window)

stock_df.filter(regex = "Target.*")

,Target 3D,Target 7D,Target 15D,Target 30D
0,2630.05,2649.40,2510.95,2367.80
1,2677.05,2610.75,2487.95,2305.80
2,2704.85,2595.65,2499.70,2285.45
3,2666.25,2564.55,2522.20,2378.50
4,2649.40,2642.50,2506.15,2367.25
...,...,...,...,...
770,1329.00,NaN,NaN,NaN
771,1341.45,NaN,NaN,NaN
772,NaN,NaN,NaN,NaN
773,NaN,NaN,NaN,NaN


### Saving processed data

In [20]:
stock_df.to_parquet(
    cnst.PROCESSED_DATA_DIR.joinpath(f'{STOCK_SYMBOL}-processed.parquet'), 
    index = False
)
stock_df

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,PE,Range,IsGreen,Is52WLow,Is52WHigh,Streak,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA,Value 7MA,Value 15MA,Value 30MA,Value 60MA,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,2021-02-02,2607.50,3129.00,2436.05,3129.00,3117.15,2684.35,3129.00,2436.05,12652036,3.396245e+10,812580,189.033,692.95,1,1,1,1,2,2,2021,1,33,1,1,3117.15,3117.15,3117.15,3117.15,692.95,692.95,692.95,692.95,2684.35,2684.35,2684.35,2684.35,12652036,12652036,12652036,12652036,33962446981,33962446981,33962446981,33962446981,812580,812580,812580,812580,2630.05,2649.40,2510.95,2367.80
1,2021-02-03,3239.00,3329.95,2831.25,2953.45,2924.25,3085.39,3329.95,2436.05,2955005,9.117329e+09,375333,177.335,498.70,0,0,1,1,3,2,2021,2,34,1,1,3020.70,3020.70,3020.70,3020.70,595.82,595.82,595.82,595.82,2884.87,2884.87,2884.87,2884.87,7803520,7803520,7803520,7803520,21539888009,21539888009,21539888009,21539888009,593956,593956,593956,593956,2677.05,2610.75,2487.95,2305.80
2,2021-02-04,2948.85,2967.95,2853.00,2866.00,2873.75,2902.67,3329.95,2436.05,476054,1.381829e+09,73464,174.272,114.95,0,0,0,2,4,2,2021,3,35,1,1,2971.72,2971.72,2971.72,2971.72,435.53,435.53,435.53,435.53,2890.80,2890.80,2890.80,2890.80,5361031,5361031,5361031,5361031,14820534938,14820534938,14820534938,14820534938,420459,420459,420459,420459,2704.85,2595.65,2499.70,2285.45
3,2021-02-05,2866.00,2900.00,2603.00,2621.00,2630.05,2694.77,3329.95,2436.05,587315,1.582680e+09,103675,159.494,297.00,0,0,0,3,5,2,2021,4,36,1,1,2886.30,2886.30,2886.30,2886.30,400.90,400.90,400.90,400.90,2841.80,2841.80,2841.80,2841.80,4167602,4167602,4167602,4167602,11511071310,11511071310,11511071310,11511071310,341263,341263,341263,341263,2666.25,2564.55,2522.20,2378.50
4,2021-02-08,2590.00,2736.00,2551.00,2700.00,2677.05,2666.44,3329.95,2436.05,507665,1.353659e+09,55033,162.344,185.00,1,0,0,1,8,2,2021,0,39,1,3,2844.45,2844.45,2844.45,2844.45,357.72,357.72,357.72,357.72,2806.72,2806.72,2806.72,2806.72,3435615,3435615,3435615,3435615,9479588872,9479588872,9479588872,9479588872,284017,284017,284017,284017,2649.40,2642.50,2506.15,2367.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770,2024-03-11,1384.45,1390.80,1363.00,1368.00,1367.85,1372.64,1679.95,981.95,49564,6.803342e+07,10005,49.345,27.80,1,0,0,1,11,3,2024,0,71,1,4,1384.21,1415.54,1432.52,1454.13,29.64,30.68,39.25,36.58,1385.05,1417.61,1436.67,1456.40,35752,48634,95234,94734,49339496,69434146,138641849,139018441,6126,7159,11886,10611,1329.00,NaN,NaN,NaN
771,2024-03-12,1367.90,1374.70,1325.05,1338.70,1337.20,1341.61,1679.95,981.95,57336,7.692242e+07,12955,48.240,49.65,0,0,0,1,12,3,2024,1,72,1,1,1375.11,1404.45,1428.82,1451.77,33.87,32.08,39.41,36.96,1375.97,1406.83,1432.99,1454.17,40624,49193,96071,93418,55664489,69656534,139646243,136947026,7474,7595,12146,10688,1341.45,NaN,NaN,NaN
772,2024-03-13,1337.20,1367.00,1303.05,1310.50,1308.65,1313.57,1679.95,981.95,83612,1.098304e+08,14624,47.210,63.95,0,0,0,2,13,3,2024,2,73,1,1,1360.89,1392.22,1424.33,1448.86,40.86,33.64,40.77,37.78,1362.40,1394.67,1428.78,1451.40,51783,50356,97602,94142,70248648,70379513,141498594,137787498,9435,7951,12407,10822,NaN,NaN,NaN,NaN
773,2024-03-14,1308.65,1352.40,1308.65,1328.00,1329.00,1337.46,1679.95,981.95,78961,1.056072e+08,15931,47.944,43.75,1,0,0,1,14,3,2024,3,74,1,1,1352.70,1384.09,1421.24,1446.35,43.00,33.70,40.83,38.26,1354.79,1386.48,1425.70,1449.10,56997,49060,98710,94914,76898424,67839371,142840100,138744439,10562,8061,12632,10994,NaN,NaN,NaN,NaN


## Data standardization

### Stock price based columns

In [21]:
stock_price_cols = [
    'Open', 'High', 'Low', 'LTP', '52W H', '52W L'
] + (
    stock_df.filter(regex = "Close.*").columns.to_list() +
    stock_df.filter(regex = "Range.*").columns.to_list() +
    stock_df.filter(regex = "VWAP.*").columns.to_list() +
    stock_df.filter(regex = "Target.*").columns.to_list()
)

stock_price_cols

['Open',
 'High',
 'Low',
 'LTP',
 '52W H',
 '52W L',
 'Close',
 'Close 7MA',
 'Close 15MA',
 'Close 30MA',
 'Close 60MA',
 'Range',
 'Range 7MA',
 'Range 15MA',
 'Range 30MA',
 'Range 60MA',
 'VWAP',
 'VWAP 7MA',
 'VWAP 15MA',
 'VWAP 30MA',
 'VWAP 60MA',
 'Target 3D',
 'Target 7D',
 'Target 15D',
 'Target 30D']

In [22]:
stock_df[stock_price_cols] = stock_df[stock_price_cols].div(stock_df['Close'], axis = 0).round(3)
stock_df[stock_price_cols]

,Open,High,Low,LTP,52W H,52W L,Close,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,0.837,1.004,0.781,1.004,1.004,0.781,1.0,1.000,1.000,1.000,1.000,0.222,0.222,0.222,0.222,0.222,0.861,0.861,0.861,0.861,0.861,0.844,0.850,0.806,0.760
1,1.108,1.139,0.968,1.010,1.139,0.833,1.0,1.033,1.033,1.033,1.033,0.171,0.204,0.204,0.204,0.204,1.055,0.987,0.987,0.987,0.987,0.915,0.893,0.851,0.789
2,1.026,1.033,0.993,0.997,1.159,0.848,1.0,1.034,1.034,1.034,1.034,0.040,0.152,0.152,0.152,0.152,1.010,1.006,1.006,1.006,1.006,0.941,0.903,0.870,0.795
3,1.090,1.103,0.990,0.997,1.266,0.926,1.0,1.097,1.097,1.097,1.097,0.113,0.152,0.152,0.152,0.152,1.025,1.081,1.081,1.081,1.081,1.014,0.975,0.959,0.904
4,0.967,1.022,0.953,1.009,1.244,0.910,1.0,1.063,1.063,1.063,1.063,0.069,0.134,0.134,0.134,0.134,0.996,1.048,1.048,1.048,1.048,0.990,0.987,0.936,0.884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770,1.012,1.017,0.996,1.000,1.228,0.718,1.0,1.012,1.035,1.047,1.063,0.020,0.022,0.022,0.029,0.027,1.004,1.013,1.036,1.050,1.065,0.972,NaN,NaN,NaN
771,1.023,1.028,0.991,1.001,1.256,0.734,1.0,1.028,1.050,1.069,1.086,0.037,0.025,0.024,0.029,0.028,1.003,1.029,1.052,1.072,1.087,1.003,NaN,NaN,NaN
772,1.022,1.045,0.996,1.001,1.284,0.750,1.0,1.040,1.064,1.088,1.107,0.049,0.031,0.026,0.031,0.029,1.004,1.041,1.066,1.092,1.109,NaN,NaN,NaN,NaN
773,0.985,1.018,0.985,0.999,1.264,0.739,1.0,1.018,1.041,1.069,1.088,0.033,0.032,0.025,0.031,0.029,1.006,1.019,1.043,1.073,1.090,NaN,NaN,NaN,NaN


### Volume based columns

In [23]:
volume_cols = stock_df.filter(regex = "Volume.*").columns.to_list()

volume_cols

['Volume', 'Volume 7MA', 'Volume 15MA', 'Volume 30MA', 'Volume 60MA']

In [24]:
stock_df[volume_cols] = stock_df[volume_cols].div(stock_df['Volume'], axis = 0).round(3)
stock_df[volume_cols]

,Volume,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA
0,1.0,1.000,1.000,1.000,1.000
1,1.0,2.641,2.641,2.641,2.641
2,1.0,11.261,11.261,11.261,11.261
3,1.0,7.096,7.096,7.096,7.096
4,1.0,6.767,6.767,6.767,6.767
...,...,...,...,...,...
770,1.0,0.721,0.981,1.921,1.911
771,1.0,0.709,0.858,1.676,1.629
772,1.0,0.619,0.602,1.167,1.126
773,1.0,0.722,0.621,1.250,1.202


### Value based columns

In [25]:
value_cols = stock_df.filter(regex = "Value.*").columns.to_list()

value_cols

['Value', 'Value 7MA', 'Value 15MA', 'Value 30MA', 'Value 60MA']

In [26]:
stock_df[value_cols] = stock_df[value_cols].div(stock_df['Value'], axis = 0).round(3)
stock_df[value_cols]

,Value,Value 7MA,Value 15MA,Value 30MA,Value 60MA
0,1.0,1.000,1.000,1.000,1.000
1,1.0,2.363,2.363,2.363,2.363
2,1.0,10.725,10.725,10.725,10.725
3,1.0,7.273,7.273,7.273,7.273
4,1.0,7.003,7.003,7.003,7.003
...,...,...,...,...,...
770,1.0,0.725,1.021,2.038,2.043
771,1.0,0.724,0.906,1.815,1.780
772,1.0,0.640,0.641,1.288,1.255
773,1.0,0.728,0.642,1.353,1.314


### Trade count based columns

In [27]:
trade_count_cols = stock_df.filter(regex = "#Trades.*").columns.to_list()

trade_count_cols

['#Trades', '#Trades 7MA', '#Trades 15MA', '#Trades 30MA', '#Trades 60MA']

In [28]:
stock_df[trade_count_cols] = stock_df[trade_count_cols].div(stock_df['#Trades'], axis = 0).round(3)
stock_df[trade_count_cols]

,#Trades,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA
0,1.0,1.000,1.000,1.000,1.000
1,1.0,1.582,1.582,1.582,1.582
2,1.0,5.723,5.723,5.723,5.723
3,1.0,3.292,3.292,3.292,3.292
4,1.0,5.161,5.161,5.161,5.161
...,...,...,...,...,...
770,1.0,0.612,0.716,1.188,1.061
771,1.0,0.577,0.586,0.938,0.825
772,1.0,0.645,0.544,0.848,0.740
773,1.0,0.663,0.506,0.793,0.690


### Dropping unnecessary columns

In [29]:
stock_df = stock_df.drop(columns = ['Date', 'Close', 'Range', 'Volume', 'Value', '#Trades'])
stock_df

,Open,High,Low,LTP,VWAP,52W H,52W L,PE,IsGreen,Is52WLow,Is52WHigh,Streak,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA,Value 7MA,Value 15MA,Value 30MA,Value 60MA,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,0.837,1.004,0.781,1.004,0.861,1.004,0.781,189.033,1,1,1,1,2,2,2021,1,33,1,1,1.000,1.000,1.000,1.000,0.222,0.222,0.222,0.222,0.861,0.861,0.861,0.861,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,0.844,0.850,0.806,0.760
1,1.108,1.139,0.968,1.010,1.055,1.139,0.833,177.335,0,0,1,1,3,2,2021,2,34,1,1,1.033,1.033,1.033,1.033,0.204,0.204,0.204,0.204,0.987,0.987,0.987,0.987,2.641,2.641,2.641,2.641,2.363,2.363,2.363,2.363,1.582,1.582,1.582,1.582,0.915,0.893,0.851,0.789
2,1.026,1.033,0.993,0.997,1.010,1.159,0.848,174.272,0,0,0,2,4,2,2021,3,35,1,1,1.034,1.034,1.034,1.034,0.152,0.152,0.152,0.152,1.006,1.006,1.006,1.006,11.261,11.261,11.261,11.261,10.725,10.725,10.725,10.725,5.723,5.723,5.723,5.723,0.941,0.903,0.870,0.795
3,1.090,1.103,0.990,0.997,1.025,1.266,0.926,159.494,0,0,0,3,5,2,2021,4,36,1,1,1.097,1.097,1.097,1.097,0.152,0.152,0.152,0.152,1.081,1.081,1.081,1.081,7.096,7.096,7.096,7.096,7.273,7.273,7.273,7.273,3.292,3.292,3.292,3.292,1.014,0.975,0.959,0.904
4,0.967,1.022,0.953,1.009,0.996,1.244,0.910,162.344,1,0,0,1,8,2,2021,0,39,1,3,1.063,1.063,1.063,1.063,0.134,0.134,0.134,0.134,1.048,1.048,1.048,1.048,6.767,6.767,6.767,6.767,7.003,7.003,7.003,7.003,5.161,5.161,5.161,5.161,0.990,0.987,0.936,0.884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770,1.012,1.017,0.996,1.000,1.004,1.228,0.718,49.345,1,0,0,1,11,3,2024,0,71,1,4,1.012,1.035,1.047,1.063,0.022,0.022,0.029,0.027,1.013,1.036,1.050,1.065,0.721,0.981,1.921,1.911,0.725,1.021,2.038,2.043,0.612,0.716,1.188,1.061,0.972,NaN,NaN,NaN
771,1.023,1.028,0.991,1.001,1.003,1.256,0.734,48.240,0,0,0,1,12,3,2024,1,72,1,1,1.028,1.050,1.069,1.086,0.025,0.024,0.029,0.028,1.029,1.052,1.072,1.087,0.709,0.858,1.676,1.629,0.724,0.906,1.815,1.780,0.577,0.586,0.938,0.825,1.003,NaN,NaN,NaN
772,1.022,1.045,0.996,1.001,1.004,1.284,0.750,47.210,0,0,0,2,13,3,2024,2,73,1,1,1.040,1.064,1.088,1.107,0.031,0.026,0.031,0.029,1.041,1.066,1.092,1.109,0.619,0.602,1.167,1.126,0.640,0.641,1.288,1.255,0.645,0.544,0.848,0.740,NaN,NaN,NaN,NaN
773,0.985,1.018,0.985,0.999,1.006,1.264,0.739,47.944,1,0,0,1,14,3,2024,3,74,1,1,1.018,1.041,1.069,1.088,0.032,0.025,0.031,0.029,1.019,1.043,1.073,1.090,0.722,0.621,1.250,1.202,0.728,0.642,1.353,1.314,0.663,0.506,0.793,0.690,NaN,NaN,NaN,NaN


### Saving standardized data

In [30]:
stock_df.to_parquet(
    cnst.PROCESSED_DATA_DIR.joinpath(f'{STOCK_SYMBOL}-standardized.parquet'), 
    index = False
)
stock_df

,Open,High,Low,LTP,VWAP,52W H,52W L,PE,IsGreen,Is52WLow,Is52WHigh,Streak,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA,Value 7MA,Value 15MA,Value 30MA,Value 60MA,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,0.837,1.004,0.781,1.004,0.861,1.004,0.781,189.033,1,1,1,1,2,2,2021,1,33,1,1,1.000,1.000,1.000,1.000,0.222,0.222,0.222,0.222,0.861,0.861,0.861,0.861,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,0.844,0.850,0.806,0.760
1,1.108,1.139,0.968,1.010,1.055,1.139,0.833,177.335,0,0,1,1,3,2,2021,2,34,1,1,1.033,1.033,1.033,1.033,0.204,0.204,0.204,0.204,0.987,0.987,0.987,0.987,2.641,2.641,2.641,2.641,2.363,2.363,2.363,2.363,1.582,1.582,1.582,1.582,0.915,0.893,0.851,0.789
2,1.026,1.033,0.993,0.997,1.010,1.159,0.848,174.272,0,0,0,2,4,2,2021,3,35,1,1,1.034,1.034,1.034,1.034,0.152,0.152,0.152,0.152,1.006,1.006,1.006,1.006,11.261,11.261,11.261,11.261,10.725,10.725,10.725,10.725,5.723,5.723,5.723,5.723,0.941,0.903,0.870,0.795
3,1.090,1.103,0.990,0.997,1.025,1.266,0.926,159.494,0,0,0,3,5,2,2021,4,36,1,1,1.097,1.097,1.097,1.097,0.152,0.152,0.152,0.152,1.081,1.081,1.081,1.081,7.096,7.096,7.096,7.096,7.273,7.273,7.273,7.273,3.292,3.292,3.292,3.292,1.014,0.975,0.959,0.904
4,0.967,1.022,0.953,1.009,0.996,1.244,0.910,162.344,1,0,0,1,8,2,2021,0,39,1,3,1.063,1.063,1.063,1.063,0.134,0.134,0.134,0.134,1.048,1.048,1.048,1.048,6.767,6.767,6.767,6.767,7.003,7.003,7.003,7.003,5.161,5.161,5.161,5.161,0.990,0.987,0.936,0.884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770,1.012,1.017,0.996,1.000,1.004,1.228,0.718,49.345,1,0,0,1,11,3,2024,0,71,1,4,1.012,1.035,1.047,1.063,0.022,0.022,0.029,0.027,1.013,1.036,1.050,1.065,0.721,0.981,1.921,1.911,0.725,1.021,2.038,2.043,0.612,0.716,1.188,1.061,0.972,NaN,NaN,NaN
771,1.023,1.028,0.991,1.001,1.003,1.256,0.734,48.240,0,0,0,1,12,3,2024,1,72,1,1,1.028,1.050,1.069,1.086,0.025,0.024,0.029,0.028,1.029,1.052,1.072,1.087,0.709,0.858,1.676,1.629,0.724,0.906,1.815,1.780,0.577,0.586,0.938,0.825,1.003,NaN,NaN,NaN
772,1.022,1.045,0.996,1.001,1.004,1.284,0.750,47.210,0,0,0,2,13,3,2024,2,73,1,1,1.040,1.064,1.088,1.107,0.031,0.026,0.031,0.029,1.041,1.066,1.092,1.109,0.619,0.602,1.167,1.126,0.640,0.641,1.288,1.255,0.645,0.544,0.848,0.740,NaN,NaN,NaN,NaN
773,0.985,1.018,0.985,0.999,1.006,1.264,0.739,47.944,1,0,0,1,14,3,2024,3,74,1,1,1.018,1.041,1.069,1.088,0.032,0.025,0.031,0.029,1.019,1.043,1.073,1.090,0.722,0.621,1.250,1.202,0.728,0.642,1.353,1.314,0.663,0.506,0.793,0.690,NaN,NaN,NaN,NaN
